In [5]:
import os
import networkx as nx
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch

In [6]:
root_dir = r"C:/Users/Rémi/Documents/stage/dataset_graph/Events"

In [7]:
dataset = []

def extract_node_features(G):
    for node, attr in G.nodes(data=True):
        try:
            # Conversion manuelle des features à float
            features = [
                float(attr.get('x', 0.0)),
                float(attr.get('y', 0.0)),
                float(attr.get('distance_to_ball', 0.0)),
                float(attr.get('gradient_x', 0.0)),
                float(attr.get('gradient_y', 0.0)),
                float(attr.get('vitesse', 0.0)),
            ]
        except Exception as e:
            print(f"Erreur dans les features du noeud {node} : {e}")
            features = [0.0] * 6

        G.nodes[node]['x'] = features
    return G

for dirpath, _, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith(".gexf"):
            full_path = os.path.join(dirpath, filename)
            try:
                G = nx.read_gexf(full_path)
                G = extract_node_features(G)
                data = from_networkx(G)

                # Vérification présence de .x
                if not hasattr(data, 'x'):
                    num_nodes = G.number_of_nodes()
                    data.x = torch.zeros((num_nodes, 6), dtype=torch.float)

                dataset.append(data)

            except Exception as e:
                print(f"Erreur lors du traitement de {full_path} : {e}")

print(f"✅ Total de graphes chargés : {len(dataset)}")


Erreur lors du traitement de C:/Users/Rémi/Documents/stage/dataset_graph/Events\AGEN_EMERGENT\Events_pos_11_t_1.34.gexf : Not all edges contain the same attributes
Erreur lors du traitement de C:/Users/Rémi/Documents/stage/dataset_graph/Events\AGEN_EMERGENT\Events_pos_11_t_14.98.gexf : Not all edges contain the same attributes
Erreur lors du traitement de C:/Users/Rémi/Documents/stage/dataset_graph/Events\AGEN_EMERGENT\Events_pos_11_t_7.04.gexf : Not all edges contain the same attributes
Erreur lors du traitement de C:/Users/Rémi/Documents/stage/dataset_graph/Events\AGEN_EMERGENT\Events_pos_11_t_9.98.gexf : Not all edges contain the same attributes
Erreur lors du traitement de C:/Users/Rémi/Documents/stage/dataset_graph/Events\AGEN_EMERGENT\Events_pos_12_t_3.0.gexf : Not all edges contain the same attributes
Erreur lors du traitement de C:/Users/Rémi/Documents/stage/dataset_graph/Events\AGEN_EMERGENT\Events_pos_13_t_0.0.gexf : Not all edges contain the same attributes
Erreur lors du tr

In [8]:
loader = DataLoader(dataset, batch_size=4, shuffle=True)

# 🧪 Test d’un batch
for batch in loader:
    print("Taille du batch :", batch.num_graphs)
    print("Features des noeuds (x) :", batch.x.shape)
    print("Edge index :", batch.edge_index.shape)
    print("Batch vector :", batch.batch.shape)
    break  # juste un batch pour vérification

KeyError: 'distance'